# BioData Catalyst Powered by PIC-SURE: Identify stigmatizing variables

The purpose of this notebook is to identify stigmatizing variables in [BioData Catalyst Powered by PIC-SURE](https://picsure.biodatacatalyst.nhlbi.nih.gov/). Specifically, stigmatizing variables will be identified in PIC-SURE Authorized Access and removed for PIC-SURE Open Access.

For more information about stigmatizing variables, please view the [README.md](https://github.com/hms-dbmi/biodata_catalyst_stigmatizing_variables#biodata_catalyst_stigmatizing_variables).

### Install packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from collections import Counter
from pprint import pprint
import json

In [2]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

  Cloning https://github.com/hms-dbmi/pic-sure-python-client.git to /tmp/pip-req-build-1foxnynx
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-client.git /tmp/pip-req-build-1foxnynx
  Created wheel for PicSureClient: filename=PicSureClient-0.1.0-py2.py3-none-any.whl size=10300 sha256=b365742553af542bb08f65cb514f03922b54e6d852a9fe65af7c550fb11e87bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-d21q2st5/wheels/31/ef/21/e362bba8de04e0072fafec9f77bd1abdf7e166213d27e98729
Successfully built PicSureClient
  Cloning https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git to /tmp/pip-req-build-9xbhj3az
  Running command git clone -q https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git /tmp/pip-req-build-9xbhj3az
     |████████████████████████████████| 95 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 67 kB 4.7 MB/s  eta 0:00:01
  Created wheel for PicSureHpdsLib: filename=PicSureHpdsLib-0.9.0-py2.py3-none-any.whl size=22051 sha256=

In [3]:
import PicSureClient
import PicSureBdcAdapter
from python_lib.utils import get_multiIndex_variablesDict, joining_variablesDict_onCol
from python_lib.stig_utils import check_simplified_name, regex_filter_out, manual_check

### Connect to PIC-SURE

In [4]:
PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140" # Be sure to use Authorized Access resource ID
token_file = "token.txt" # Be sure to use developer token to get all variables

In [5]:
with open(token_file, "r") as f:
    my_token = f.read()

In [6]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureBdcAdapter.Adapter(connection)
resource = adapter.useResource(resource_id)



+=========================================================================================+
|        [ WARNING ] you are specifying that you WANT to allow self-signed SSL            |
|        certificates to be acceptable for connections.  This may be useful for           |
|        working in a development environment or on systems that host public              |
|        data.  BEST SECURITY PRACTICES ARE THAT IF YOU ARE WORKING WITH SENSITIVE        |
|        DATA THEN ALL SSL CERTS BY THOSE EVIRONMENTS SHOULD NOT BE SELF-SIGNED.          |
+=========================================================================================+

+--------------------------------------+------------------------------------------------------+
|  Resource UUID                       |  Resource Name                                       |
+--------------------------------------+------------------------------------------------------+
| 02e23f52-f354-4e8b-992c-d37c8b9ba140 |                         

### Save all variables in PIC-SURE Authorized Access to DataFrame

In [7]:
fullVariableDict = resource.dictionary().find().DataFrame()
#fullVariableDict
multiindex = get_multiIndex_variablesDict(fullVariableDict)

In [ ]:
multiindex # potentially explore categoryValues

### Identify stigmatizing variables using `simplified_name`

There are two functions to identify stigmatizing variables: `check_simplified_name` and `regex_filter_out`. 

`check_simplified_name` selects all variables from the `multiindex` dataframe where the `simplified_name` contains any of the terms in the given list. It also takes an optional argument `exclude_vars` that removes variable with specified `simplified_name` variables.

For example, 

`check_simplified_name(['bio', 'data', 'catalyst'], multiindex, ['biology variable'])`

would find all variables where the `simplified_name` contains 'bio', 'data', and/or 'catalyst' but excludes `simplified_name`s equal to 'biology variable' (ignoring capitalization).

`regex_filter_out` uses the list of potentially stigmatizing variables and filters out any `simplified_name` variables containing the given list of terms. Unlike `check_simplified_name` where excluded variables must match the `simplified_name` completely, this function excludes the variable if the term is *contained* by the `simplified_name`. 

For example,

`regex_filter_out(['biodata catalyst', 'terra', 'heliobacter pylori'], ['ter'])`

would exclude all variables containing '*ter*'. In this case, '*ter*ra' and 'heliobac*ter* pylori' would be removed.

| Function | Arguments / Input | Output|
|--------|-------------------|-------|
| check_simplified_name() | (1) list of search terms, (2) multiindex dataframe, (3) optional: variables to exclude | (1) list of potentially stigmatizing variables, (2) variables excluded using provided criteria|
| regex_filter_out() | (1) list of stigmatizing variables, (2) list of terms to filter | list of stigmatizing variables that do not contain any of terms to filter |

#### Sex history filtering
The following terms are used to filter out sex history variables:

<table border="0">
    <tr>
        <td>sex</td>
        <td>sex history</td>
    </tr>
    <tr>
        <td>sexual</td>
        <td>sexually</td>
    </tr>
    <tr>
        <td>intercourse</td>
        <td>coitus</td>
    </tr>
    <tr>
        <td>copulation</td>
        <td>pareunia</td>
    </tr>
    <tr>
        <td>futunio</td>
        <td>venery</td>
    </tr>
</table>

The following `simplified_name` variables are excluded:
<table border="0">
    <tr>
        <td>sex</td>
    </tr>
    <tr>
        <td>sex of participant</td>
    </tr>
</table>

`simplified_name` variables containing the following terms are excluded:
<table border="0">
    <tr>
        <td>race and sex adjusted</td>
    </tr>
</table>

In [ ]:
sex_history_terms = ['sex', 'sex history', 'sexual', 'sexually', 'intercourse', 
                     'coitus', 'copulation', 'pareunia', 'futunio', 'venery']
sex_remove = ['sex', 'sex of participant']
terms_to_filter = ['race and sex adjusted']

In [ ]:
sex_stig_vars, ex_sex_vars = check_simplified_name(sex_history_terms, multiindex, exclude_vars=sex_remove)

In [ ]:
final_sex_vars = regex_filter_out(sex_stig_vars, terms_to_filter)

In [ ]:
print("Total number of sex vars", len(sex_stig_vars))
print("After filtering", len(final_sex_vars))

#### Sexually transmitted disease diagnosis/history/treatment filtering
The following terms are used to filter out variables related to sexually transmitted disease:

<table border="0">
    <tr>
        <td>chlamydia</td>
        <td>genital</td>
    </tr>
    <tr>
        <td>herpes</td>
        <td>gonorrhea</td>
    </tr>
    <tr>
        <td>HIV</td>
        <td>AIDS</td>
    </tr>
    <tr>
        <td>pubic lice</td>
        <td>syphilis</td>
    </tr>
    <tr>
        <td>trichomoniasis</td>
        <td>vagina</td>
    </tr>
    <tr>
        <td>progesterone</td>
        <td>estrogen</td>
    </tr>
</table>

`simplified_name` variables containing the following terms are excluded:
<table border="0">
    <tr>
        <td>hives</td>
        <td>health aids</td>
    </tr>
    <tr>
        <td>nsaids</td>
        <td>herpes zoster</td>
    </tr>
    <tr>
        <td>chlamydia pneumoniae</td>
        <td>heart disease</td>
    </tr>
    <tr>
        <td>walking aid</td>
        <td>archive</td>
    </tr>
    <tr>
        <td>shiver</td>
    </tr>
</table>

In [8]:
sex_disease_terms = ['chlamydia', 'genital', 'herpes', 'gonorrhea', 'hiv', 
                     'aids', 'hpv', 'pubic lice', 'syphilis', 'trichomoniasis', 
                     'estrogen', 'vagina', 'progesterone', "venereal", "penis", 
                     "vagina", "antiviral"]
terms_to_filter = ['hives', 'health aids', 'nsaids', 'herpes zoster', 'chlamydia pneumoniae', 
                   'heart disease', 'walking aid', 'archive', 'shiver']

In [9]:
sex_disease_stig_vars, ex_sex_disease_vars = check_simplified_name(sex_disease_terms, multiindex)

In [10]:
final_sex_disease_vars = regex_filter_out(sex_disease_stig_vars, terms_to_filter)

In [11]:
print("Total number of sex disease vars", len(sex_disease_stig_vars))
print("After filtering", len(final_sex_disease_vars))

Total number of sex disease vars 1095
After filtering 940


#### Mental health diagnoses/history/treatment filtering
The following terms are used to filter out variables related to mental health:

<table border="0">
    <tr>
        <td>depression</td>
        <td>depressive</td>
    </tr>
    <tr>
        <td>anxiety</td>
        <td>panic</td>
    </tr>
    <tr>
        <td>phobia</td>
        <td>schizophrenia</td>
    </tr>
    <tr>
        <td>mental</td>
        <td>mental health</td>
    </tr>
    <tr>
        <td>psycho</td>
        <td>psychological</td>
    </tr>
    <tr>
        <td>emotional health</td>
        <td></td>
    </tr>
</table>

`simplified_name` variables containing the following terms are excluded:
<table border="0">
    <tr>
        <td>hispanic</td>
        <td>electrocardiograph</td>
    </tr>
    <tr>
        <td>minn code</td>
        <td>minnesota code</td>
    </tr>
    <tr>
        <td>ecg</td>
        <td>environmental</td>
    </tr>
    <tr>
        <td>instrumental</td>
        <td>mini-mental state exam</td>
    </tr>
    <tr><td>supplemental</td><td></td></tr>
</table>

In [8]:
mental_health_terms = ['depression', 'depressive', 'anxiety', 'panic', 'phobias', 'schizophrenia',
                       'mental', 'mental health', 'psycho', 'psychological', 'emotional health']
terms_to_filter = ['hispanic', 'electrocardiograph', 'minn code', 'minnesota code', 
                   'ecg', 'environmental', 'instrumental', 'supplemental', 'mini-mental state exam']

In [9]:
mental_health_stig_vars, ex_mental_health_vars = check_simplified_name(mental_health_terms, multiindex)

In [10]:
final_mental_health_vars = regex_filter_out(mental_health_stig_vars, terms_to_filter)

In [11]:
print("Total number of mental health vars", len(mental_health_stig_vars))
print("After filtering", len(final_mental_health_vars))

Total number of mental health vars 985
After filtering 717


#### Illicit drug use history filtering
The following terms are used to filter out variables related to illicit drug use:

<table border="0">
    <tr>
        <td>illicit</td>
        <td>street drug</td>
        <td>rohypnol</td>
    </tr>
    <tr>
        <td>abuse</td>
        <td>illegal</td>
        <td>roofies</td>
    </tr>
    <tr>
        <td>fentanyl</td>
        <td>cocaine</td>
        <td>ketamine hydrochloride</td>
    </tr>
    <tr>
        <td>ecstasy</td>
        <td>LSD</td>
        <td>psilocybin</td>
    </tr>
    <tr>
        <td>methamphetamine</td>
        <td>heroin</td>
        <td>mushroom</td>
    </tr>
    <tr>
        <td>phencyclidine</td>
        <td>angel dust</td>
        <td>krokodil</td>
    </tr>
    <tr>
        <td>mushroom</td>
        <td>salvia</td>
        <td>bath salts</td>
    </tr>
    <tr>
        <td>flakka</td>
        <td>ayahuasca</td>
        <td>DMT</td>
    </tr>
    <tr>
        <td>central nervous system depressant</td>
        <td>hallucinogen</td>
        <td>inhalant</td>
    </tr>
    <tr>
        <td>khat</td>
        <td>kratom</td>
        <td>mescaline</td>
    </tr>
    <tr>
        <td>loperamide</td>
        <td>dextromethorphan</td>
        <td>opioid</td>
    </tr>
    <tr>
        <td>stimulant</td>
        <td>cannabinoid</td>
        <td>gamma hydroxybutyrate</td>
    </tr>
</table>

`simplified_name` variables containing the following terms are excluded:
<table border="0">
    <tr>
        <td>coffee or tea</td>
    </tr>
</table>

In [14]:
illicit_drug_terms = ['illicit', 'street drug', 'abuse', 'illegal', 'fentanyl', 
                      'cocaine', 'ecstasy', 'lsd', 'methamphetamine', 'heroin', 
                      'phencyclidine', 'angel dust', 'rohypnol', 'roofies', 
                      'ketamine hydrochloride', 'psilocybin', 'mushroom', 'krokodil', 
                      'marijuana', 'salvia','bath salts', 'flakka', 'ayahuasca', 'dmt', 
                      'central nervous system depressant', 'hallucinogen', 'inhalant', 'khat', 
                      'kratom', 'mescaline', 'loperamide', 'dextromethorphan','opioid', 
                      'stimulant', 'cannabinoid', 'gamma hydroxybutyrate', 'depressants']
terms_to_filter = ['coffee or tea']

In [15]:
illicit_drug_stig_vars, ex_illicit_drug_vars = check_simplified_name(illicit_drug_terms, multiindex)

In [16]:
final_illicit_drug_vars = regex_filter_out(illicit_drug_stig_vars, terms_to_filter)

In [17]:
print("Total number of illicit drug vars", len(illicit_drug_stig_vars))
print("After filtering", len(final_illicit_drug_vars))

Total number of illicit drug vars 136
After filtering 127


#### Intellectual achievement/ability/educational attainment filtering
The following terms are used to filter out variables related to intellectual achievement:
(Note from Rui: no genetics IQ outcomes)

<table border="0">
    <tr>
        <td>bachelor</td>
        <td>master</td>
    </tr>
    <tr>
        <td>phd</td>
        <td>quotient</td>
    </tr>
    <tr>
        <td>intellectual</td>
        <td>intelligence</td>
    </tr>
    <tr>
        <td>acheivement</td>
        <td>disability</td>
    </tr>
    <tr>
        <td>ability</td>
        <td>attainment</td>
    </tr>
    <tr>
        <td>education</td>
        <td>genetic iq</td>
    </tr>
    <tr>
        <td>school</td>
        <td></td>
    </tr>
</table>

`simplified_name` variables containing the following terms are excluded:
<table border="0">
    <tr>
        <td>change in ability to</td>
        <td>how ability to</td>
    </tr>
    <tr>
        <td>ability to</td>
        <td>variability</td>
    </tr>
    <tr>
        <td>gradability</td>
        <td>reliability</td>
    </tr>
    <tr>
        <td>acceptability</td>
        <td>irritability</td>
    </tr>
    <tr>
        <td>leg ability</td>
        <td>physical ability</td>
    </tr>
</table>

In [20]:
intell_ability_terms = ['bachelor', 'master', 'phd', 'quotient', 'intellectual', 'intelligence',
                        'achievement', 'disability', 'ability', 'attainment', 'education', 'genetic iq', 'school']
terms_to_filter = ['change in ability to', 'how ability to', 'ability to', 
                   'variability', 'gradability', 'reliability', 'acceptability', 
                   'irritability', 'leg ability', 'physical ability']

In [21]:
intell_ability_stig_vars, ex_intell_ability_vars = check_simplified_name(intell_ability_terms, multiindex)

In [22]:
final_intell_ability_vars = regex_filter_out(intell_ability_stig_vars, terms_to_filter)

In [23]:
print("Total number of intellectual vars", len(intell_ability_stig_vars))
print("After filtering", len(final_intell_ability_vars))

Total number of intellectual vars 412
After filtering 212


#### Direct or surrogate identifiers of legal status filtering
The following terms are used to filter out variables related to legal status:

<table border="0">
    <tr>
        <td>villainage</td>
        <td>villeinage</td>
    </tr>
    <tr>
        <td>citizenship</td>
        <td>marital</td>
    </tr>
    <tr>
        <td>married</td>
        <td>unmarried</td>
    </tr>
    <tr>
        <td>single</td>
        <td>divorces</td>
    </tr>
    <tr>
        <td>widowed</td>
        <td>minority</td>
    </tr>
    <tr>
        <td>nonage</td>
        <td>marriage</td>
    </tr>
    <tr>
        <td>matrimony</td>
        <td>spousal</td>
    </tr>
    <tr>
        <td>civil union</td>
        <td>wedlock</td>
    </tr>
    <tr>
        <td>bachelorhood</td>
        <td>spinsterhood</td>
    </tr>
    <tr>
        <td>widowhood</td>
        <td>ethnicity</td>
    </tr>
    <tr>
        <td>nationality</td>
        <td>race</td>
    </tr>
    <tr>
        <td>death</td>
        <td>identifier</td>
    </tr>
    <tr>
        <td>identity</td>
        <td>surrogate</td>
    </tr>
    <tr>
        <td>legal status</td>
        <td></td>
    </tr>
</table>

`simplified_name` variables containing the following terms are excluded:
<table border="0">
    <tr>
        <td>single tennis</td>
        <td>single ventricular</td>
    </tr>
    <tr>
        <td>single nodule</td>
        <td>urinalysis: albumin</td>
    </tr>
    <tr>
        <td>brace</td>
        <td>contraceptive</td>
    </tr>
    <tr>
        <td>race and sex adjusted</td>
        <td>single sup</td>
    </tr>
</table>

In [ ]:
legal_status_terms = ['villainage', 'villeinage', 'citizenship', 'marital', 
                      'married', 'unmarried', 'single', 'divorced', 'widowed', 
                      'minority', 'nonage', 'marriage', 'matrimony', 'spousal',  
                      'civil union', 'wedlock', 'bachelorhood', 'spinsterhood',
                      'widowhood', 'ethnicity', 'nationality', 'race', 'death', 
                      'identifier', 'identity', 'surrogate', 'legal status']
#legal_status_remove = ['subject identifier']
terms_to_filter = ['single tennis', 'single ventricular', 'single nodule', 
                   'urinalysis: albumin', 'brace', 'contraceptive', 
                   'race and sex adjusted', 'single sup']

In [ ]:
legal_status_stig_vars, ex_legal_status_vars = check_simplified_name(legal_status_terms, multiindex)

In [ ]:
final_legal_status_vars = regex_filter_out(legal_status_stig_vars, terms_to_filter)

In [ ]:
print("Total number of legal status vars", len(legal_status_stig_vars))
print("After filtering", len(final_legal_status_vars))

### Manual review of potentially stigmatizing variables

`manual_check` provides an interactive way to record whether filtered variables are indeed stigmatizing. It uses the list of stigmatizing variables and also takes an optional argument `ex_vars` that provides a manual review of the excluded terms. A dataframe of the stigmatizing variables with recorded responses and (if applicable) a dataframe of excluded variables and recorded responses are returned.

To use this function, simply call it on the list of filtered variables (and excluded variables if needed) and follow the interactive instructions.

In [ ]:
sex_stigs, ex_sex_stigs = manual_check(final_sex_vars)

In [12]:
sex_disease_stigs, ex_sex_disease_stigs = manual_check(final_sex_disease_vars)

Continue to review of 940 variables?
y/n: y
Is the following variable stigmatizing?  1 of 940

>>>> Estrogens, excluding vaginal creams <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  2 of 940

>>>> HAVE YOU EVER TAKEN OTHER ESTROGENS <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  3 of 940

>>>> Premarin (conjugated estrogens) <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  4 of 940

>>>> AGE STOPPED TAKING ESTROGEN <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  5 of 940

>>>> CURRENT ESTROGEN DAYS/MONTH <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  6 of 940

>>>> CURRENTLY TAKING ESTROGEN <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigma

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  54 of 940

>>>> TOOK ESTROGEN TO PREVENT OSTEOPOROSIS <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  55 of 940

>>>> TOOK ESTROGEN TO PREVENT OTHER SYMPTOMS <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  56 of 940

>>>> YEAR STOPPED TAKING ESTROGEN <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  57 of 940

>>>> YEARS CURRENTLY TAKING ESTROGEN <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  58 of 940

>>>> YEARS TOOK ESTROGEN IN PAST <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  59 of 940

>>>> Calculated estrogen use at bl <<<<

Type "yes" or "no". To display full variable, type "more":


Is the following variable stigmatizing?  100 of 940

>>>> FEMALE HORMONE REPLACEMENT: PROGESERONE PREPARATION: STRENGTH CHARACTER VARIABLE; (BLANK) = UNKNOWN, OR NO PROGESTERONE PREPARATION USE (207) <<<<

Type "yes" or "no". To display full variable, type "more":
more

>>>> \Framingham Cohort ( phs000007 )\Clinic Questionnaire (Interview and Physical Exam)\Clinic Exam Questionnaire\MD Interview, Physical Exam, Examiner's Opinion, and Clinical Diagnostic Impression; Non-MD / Non-medical Interview / Self-report and Physical Exam / Anthropometrics / Observed Performance\FEMALE HORMONE REPLACEMENT: PROGESERONE PREPARATION: STRENGTH CHARACTER VARIABLE; (BLANK) = UNKNOWN, OR NO PROGESTERONE PREPARATION USE (207)\ <<<<

Type "yes" or "no": 
y
Is the following variable stigmatizing?  101 of 940

>>>> FEMALE HORMONE REPLACEMENT: PROGESTERONE PREPARATION: NUMBER OF DAYS PER MONTH TAKEN <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  10

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  140 of 940

>>>> Medical History - Female Genitourinary Disease: Number of days per month taking estrogens <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  141 of 940

>>>> Medical History - Female Genitourinary Disease: Oral dose per day of premarin or conjugated Estrogens <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  142 of 940

>>>> Medical History - Female Genitourinary Disease: Patch dose per day of estrogen <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  143 of 940

>>>> Medical History - Female Genitourinary Disease: Progesterone use in interim <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  144 of 940

>>>> Medical History - Female Reproduct

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  182 of 940

>>>> DOSE/DAY PREMARIN OR CONJ. ESTROGENS <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  183 of 940

>>>> ECHO: OTHER CONGENITAL ABNORMALITY <<<<

Type "yes" or "no". To display full variable, type "more":
more

>>>> \Framingham Cohort ( phs000007 )\Lab Work\Blood\Hematologic\ECHO: OTHER CONGENITAL ABNORMALITY\ <<<<

Type "yes" or "no": 
n
Is the following variable stigmatizing?  184 of 940

>>>> ESTROGEN CREAM USE DURING INTERIM <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  185 of 940

>>>> ESTROGEN CREAM USE IN INTERIM - FEMALE <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  186 of 940

>>>> ESTROGEN CREAM USE IN INTERIM <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is th

...

Is the following variable stigmatizing?  720 of 940

>>>> Used estrogen pills <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  721 of 940

>>>> Used estrogen skin patch <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  722 of 940

>>>> Used estrogen vaginally <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  723 of 940

>>>> Used progesterone pills <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  724 of 940

>>>> Did you use an estrogen cream? <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  725 of 940

>>>> Did you use an estrogen patch? <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  726 of 940

>>>> Did you use any Estrogen pill <<<<

T

Is the following variable stigmatizing?  771 of 940

>>>> Natural/Progesterone cream <<<<

Type "yes" or "no". To display full variable, type "more":
more

>>>> \Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )\Form 147 - Observational Study (OS) Follow-Up Questionnaire (Year 7)\Natural/Progesterone cream\ <<<<

Type "yes" or "no": 
y
Is the following variable stigmatizing?  772 of 940

>>>> Other/ non-estrogen <<<<

Type "yes" or "no". To display full variable, type "more":
more

>>>> \Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )\Form 147 - Observational Study (OS) Follow-Up Questionnaire (Year 7)\Other/ non-estrogen\ <<<<

Type "yes" or "no": 
y
Is the following variable stigmatizing?  773 of 940

>>>> Progesterone pill or capsule <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  774 of 940

>>>> Progesterone pill, cream shot <<<<

Type "yes" or "no". To display f

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  821 of 940

>>>> Hormone use last year (estrogen or progesterone) <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  822 of 940

>>>> Kind of Oral Estrogen pills taken <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  823 of 940

>>>> Kind of Progesterone pill taken <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  824 of 940

>>>> Months had Estrogen shots <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  825 of 940

>>>> Months had Progesterone shots <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  826 of 940

>>>> Months used Estrogen skin cream <<<<

Type "yes" or "no". To display full variable, t

Is the following variable stigmatizing?  872 of 940

>>>> Age last used unopposed estrogen <<<<

Type "yes" or "no". To display full variable, type "more":
more

>>>> \Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )\Form 43 - Hormone Use Interview - current and past hormone replacement (duration, frequency); history of oral contraceptive, diethylstilbestrol, depo-provera use. This form was required for all OS participants.\Age last used unopposed estrogen\ <<<<

Type "yes" or "no": 
y
Is the following variable stigmatizing?  873 of 940

>>>> Estrogen + progesterone duration by category <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  874 of 940

>>>> Estrogen + progesterone usage status <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  875 of 940

>>>> Estrogen + progesterone use ever <<<<

Type "yes" or "no". To display full variable, type

Type "yes" or "no". To display full variable, type "more":
n
Is the following variable stigmatizing?  920 of 940

>>>> Progesterone receptor assay <<<<

Type "yes" or "no". To display full variable, type "more":
more

>>>> \Women's Health Initiative Clinical Trial and Observational Study ( phs000200 )\Outcomes - Breast Cancer (Form 122/130)\Progesterone receptor assay\ <<<<

Type "yes" or "no": 
n
Is the following variable stigmatizing?  921 of 940

>>>> Days enrollment to Genital Organs Cancer <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  922 of 940

>>>> Days enrollment to Vagina Cancer <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  923 of 940

>>>> Genital Organs Cancer ascertainment source <<<<

Type "yes" or "no". To display full variable, type "more":
y
Is the following variable stigmatizing?  924 of 940

>>>> Genital Organs Cancer <<<<

Type "yes" or "no". T

In [12]:
mental_health_stigs, ex_mental_health_stigs = manual_check(final_mental_health_vars)

Continue to review of 717 variables?
y/n: y
Is the following variable stigmatizing?  1 of 717

>>>> Anti-psychotic medications <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  2 of 717

>>>> Anti-psychotic <<<<

Type "y" or "n". To display full variable, type "more":
y
Already identified > Anti-psychotic medications <, recording result 3 of 717
Is the following variable stigmatizing?  4 of 717

>>>> DEPRESSION SCALE TOTAL <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  5 of 717

>>>> ST DEPRESSION GE 100, CH 1 (DURATION) <<<<

Type "y" or "n". To display full variable, type "more":
n
Is the following variable stigmatizing?  6 of 717

>>>> ST DEPRESSION GE 100, CH 1 (NUMBER) <<<<

Type "y" or "n". To display full variable, type "more":
n
Is the following variable stigmatizing?  7 of 717

>>>> ST DEPRESSION GE 100, CH 2 (DURATION) <<<<

Type "y" or "n". To display full variabl

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  51 of 717

>>>> CLINICAL DIAGNOSTIC IMPRESSION: ANXIETY <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  52 of 717

>>>> CLINICAL DIAGNOSTIC IMPRESSION: DEPRESSION <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  53 of 717

>>>> CLINICAL DIAGNOSTIC IMPRESSION: PSYCHOSIS <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  54 of 717

>>>> Clinical Diagnostic Impression - Part II: Non Cardiovascular Diagnoses First Examiner Opinions: Mental Health: Anxiety <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  55 of 717

>>>> Clinical Diagnostic Impression - Part II: Non Cardiovascular Diagnoses First Examiner Opinions: Mental Health: Depression <<<<

Type "y" or "n". To display f

Is the following variable stigmatizing?  96 of 717

>>>> FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: PARKINSONISM <<<<

Type "y" or "n". To display full variable, type "more":
more

>>>> \Framingham Cohort ( phs000007 )\Clinic Questionnaire (Interview and Physical Exam)\Clinic Exam Questionnaire\MD Interview, Physical Exam, Examiner's Opinion, and Clinical Diagnostic Impression; Non-MD / Non-medical Interview / Self-report and Physical Exam / Anthropometrics / Observed Performance\FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: PARKINSONISM\ <<<<

Type "y" or "n": 
y
Is the following variable stigmatizing?  97 of 717

>>>> FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: POOR EYESIGHT <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  98 of 717

>>>> FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: POOR HEARING <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizin

Already identified > MMSE: OTHER (OTHER THAN THOSE LISTED ABOVE) (FACTORS POTENTIALLY AFFECTING MENTAL STATUS) <, recording result 321 of 717
Already identified > MMSE: PARKINSONISM OR NEUROLOGICALLY IMPAIRED (FACTORS POTENTIALLY AFFECTING MENTAL STATUS) <, recording result 322 of 717
Already identified > MMSE: POOR EYESIGHT (FACTORS POTENTIALLY AFFECTING MENTAL STATUS) <, recording result 323 of 717
Already identified > MMSE: POOR HEARING (FACTORS POTENTIALLY AFFECTING MENTAL STATUS) <, recording result 324 of 717
Already identified > Medical History - Non-cardiovascular Medications II - Interim Medications: Anti-anxiety, Sedative/Hypnotics etc. (Librium, Valium, etc.) <, recording result 325 of 717
Already identified > NON-CARDIOVASCULAR MEDICATIONS: ANTI-ANXIETY, SEDATIVE/ HYPNOTICS (LIBRIUM, VALIUM, ETC.) <, recording result 326 of 717
Already identified > R IN V5 OR V6: S IN V1 OR V2: ST DEPRESSION <, recording result 327 of 717
Already identified > CLINICAL DIAGNOSTIC IMPRESSION-

Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: DEPRESSION <, recording result 509 of 717
Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: ILLITERACY OR LOW EDUCATION <, recording result 510 of 717
Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: NOT FLUENT IN ENGLISH <, recording result 511 of 717
Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: OTHER <, recording result 512 of 717
Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: PARKINSONISM <, recording result 513 of 717
Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: POOR EYESIGHT <, recording result 514 of 717
Already identified > FACTORS POTENTIALLY AFFECTING MENTAL STATUS TESTING: POOR HEARING <, recording result 515 of 717
Already identified > HOSPITALIZED SINCE LAST EXAM FOR MENTAL OR EMOTIONAL DISEASE <, recording result 516 of 717
Already identified > MEDICATION USE - ANTI-AN

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  561 of 717

>>>> Q2. A. Depression. In the past 2 weeks, have you been much less interested in most things less able to enjoy the things you used to enjoy most of the time? [Visit 9] [Major Depressive Episode Form, MDE] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  562 of 717

>>>> Q26h. In the past year did you or anyone living in your household receive an income from the following source? Supplemental security income (SSI)? [Visit 9] [Personal Data - Socioeconomic Status, PDS] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  563 of 717

>>>> Q27h. Receive supplemental security income in past year [Visit 1] [Personal Data - Socioeconomic Status, PDS] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  564 of 717

>>>> Q3a.

Is the following variable stigmatizing?  590 of 717

>>>> Supplemental aberrant supraventricular complexes, by visual flag <<<<

Type "y" or "n". To display full variable, type "more":
more

>>>> \Jackson Heart Study (JHS) Cohort ( phs000286 )\Supplemental aberrant supraventricular complexes, by visual flag\ <<<<

Type "y" or "n": 
n
Is the following variable stigmatizing?  591 of 717

>>>> Supplemental ectopic ventricular complexes, by visual flag [Visit 9] <<<<

Type "y" or "n". To display full variable, type "more":
n
Is the following variable stigmatizing?  592 of 717

>>>> ANATOMIC DIRECTION OF LEFT SUBSUPERIOR SEGMENTAL BRONCHUS <<<<

Type "y" or "n". To display full variable, type "more":
n
Is the following variable stigmatizing?  593 of 717

>>>> ANATOMIC DIRECTION OF RIGHT SUBSUPERIOR SEGMENTAL BRONCHUS <<<<

Type "y" or "n". To display full variable, type "more":
n
Is the following variable stigmatizing?  594 of 717

>>>> ANTI-PSYCHOTIC MEDICATIONS <<<<

Type "y" or "n". To d

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  632 of 717

>>>> Q3. During the past week: I felt everything I did was an effort [CES Depression Form] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  633 of 717

>>>> Q4. During the past week: my sleep was restless [CES Depression Form] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  634 of 717

>>>> Q5. During the past week: I was happy [CES Depression Form] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  635 of 717

>>>> Q6. During the past week: I felt lonely [CES Depression Form] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  636 of 717

>>>> Q7. During the past week: people were unfriendly [CES Depression Form] <<<<

Type "y" or "n". To display full variable

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  677 of 717

>>>> SF-36 Mental Health Index (0-100) (mh) <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  678 of 717

>>>> SF-36 RAW MENTAL HEALTH INDEX <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  679 of 717

>>>> SF-36 STANDARDIZED MENTAL COMPONENT SCALE-00 <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  680 of 717

>>>> SF-36 raw - Mental Health Index (raw -mh) <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  681 of 717

>>>> Lost weight due to depression <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  682 of 717

>>>> 5lbs not on purpose/Depression <<<<

Type "y" or "n". To display full variable, type "mo

In [18]:
illicit_drug_stigs, ex_illicit_drug_stigs = manual_check(final_illicit_drug_vars)

Continue to review of 127 variables?
y/n: y
Is the following variable stigmatizing?  1 of 127

>>>> bl_hospadmtm <<<<

Type "y" or "n". To display full variable, type "more":
more

>>>> \COVID19-ORCHID ( phs002299 )\bl_hospadmtm\ <<<<

Type "y" or "n": 
n
Is the following variable stigmatizing?  2 of 127

>>>> Non-tricyclic antidepressants other than <<<<

Type "y" or "n". To display full variable, type "more":
more

>>>> \Cardiovascular Health Study (CHS) Cohort: an NHLBI-funded observational study of risk factors for cardiovascular disease in adults 65 years or older ( phs000287 )\Data contain extensive medical history information of subjects (all > 65 years of age)\Non-tricyclic antidepressants other than\ <<<<

Type "y" or "n": 
y
Is the following variable stigmatizing?  3 of 127

>>>> Tri-cyclic anti-depressants plus anti-ps <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  4 of 127

>>>> Tricyclic anti-depressants <<<<

Type 

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  99 of 127

>>>> 6. Used crack or cocaine in any form [Visit 1] [Health Practices: Alcohol and Drug Use, Version A] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  100 of 127

>>>> 7. About how many times in your lifetime have you used crack or cocaine [Visit 9] [Health Practices: Alcohol and Drug Use] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  101 of 127

>>>> 7. How many times have you used crack/cocaine in your life [Visit 1] [Health Practices: Alcohol and Drug Use, Version A] <<<<

Type "y" or "n". To display full variable, type "more":
y
Is the following variable stigmatizing?  102 of 127

>>>> 8. Since your JHS Exam 1 have you ever used any other kinds of drugs including marijuana, heroine, or others? [Visit 9] [Health Practices: Alcohol and Drug Use] <<<<

Type "y" 

In [ ]:
intell_ability_drug_stigs, ex_intell_ability_stigs = manual_check(final_illicit_drug_vars)

### Export potentially stigmatizing variables

In [ ]:
sex_stigs.to_csv("sexual_health_stig_vars.tsv", sep='\t')

In [13]:
sex_disease_stigs.to_csv("sexual_disease_stig_vars.tsv", sep='\t')

In [13]:
mental_health_stigs.to_csv("mental_health_stig_vars.tsv", sep='\t')

In [19]:
illicit_drug_stigs.to_csv("illicit_drug_stig_vars.tsv", sep='\t')